# 🩺 Smart Health Assistant (Sequential Conversation)

In this section, we build a **multi‑agent health assistant** using AutoGen with a **sequential conversation pattern**:
1. **User Proxy Agent** collects user data
2. **BMI Tool** calculates BMI from height/weight
3. **BMI Agent** provides health recommendations
4. **Diet Planner Agent** creates a meal plan
5. **Workout Scheduler Agent** builds a weekly workout plan

This flow ensures each agent specializes in its step and passes context to the next.


In [1]:
import autogen
import os
from IPython import get_ipython
from autogen import ConversableAgent, AssistantAgent, UserProxyAgent
from autogen.coding import LocalCommandLineCodeExecutor
from IPython.display import HTML, display

# Core imports:
# - autogen: multi-agent orchestration framework
# - LocalCommandLineCodeExecutor: runs code safely in a local working folder
# - IPython display utilities: useful for rich notebook output

In [2]:
from dotenv import load_dotenv

# Load environment variables (e.g., API keys) from a .env file.
# If your .env lives elsewhere, pass an explicit path to load_dotenv("/path/to/.env").
load_dotenv()

True

In [9]:
# ==========================================================================
# STEP: BUILD A SMART HEALTH ASSISTANT (SEQUENTIAL CHAT)
# ==========================================================================
# This pipeline collects user data, computes BMI, and produces a full plan.

from autogen import register_function, GroupChat, GroupChatManager

# LLM config shared by assistant agents
health_llm_config = {
    "config_list": [
        {
            "model": "gpt-4o-mini",
            "temperature": 0.2,
        }
    ]
}

# 1) User Proxy Agent (data collection)
health_user_proxy = UserProxyAgent(
    name="HealthUserProxy",
    llm_config=False,
    human_input_mode="NEVER",  # Set to "ALWAYS" if you want interactive input
    code_execution_config=False,
)

# 2) BMI Agent (registered with BMI tool)
bmi_agent = AssistantAgent(
    name="BMIAgent",
    system_message=(
        "You are a BMI expert. Call calculate_bmi exactly once, then classify the BMI and "
        "give concise health recommendations. Do not ask follow-up questions. End with TERMINATE."
    ),
    llm_config=health_llm_config,
)

# 3) Diet Planner Agent
meal_planner_agent = AssistantAgent(
    name="DietPlannerAgent",
    system_message=(
        "You are a diet planner. Use BMI insights and dietary preference to suggest a "
        "simple 7-day meal plan. Keep it practical and balanced. Do not ask follow-up questions. "
        "End with TERMINATE."
    ),
    llm_config=health_llm_config,
)

# 4) Workout Scheduler Agent
workout_scheduler_agent = AssistantAgent(
    name="WorkoutSchedulerAgent",
    system_message=(
        "You are a fitness coach. Read the full conversation context and produce the FINAL health plan "
        "containing three sections: BMI insights, diet plan, and weekly workout schedule. "
        "Include rest days. Do not ask follow-up questions. End with TERMINATE."
    ),
    llm_config=health_llm_config,
)

# BMI Tool
def calculate_bmi(height_cm: float, weight_kg: float) -> dict:
    """Calculate BMI from height (cm) and weight (kg)."""
    height_m = height_cm / 100.0
    bmi = weight_kg / (height_m ** 2)
    return {
        "height_m": round(height_m, 2),
        "bmi": round(bmi, 2),
    }

# Register BMI tool with the BMI Agent (caller) and User Proxy (executor)
register_function(
    calculate_bmi,
    caller=bmi_agent,
    executor=health_user_proxy,
    name="calculate_bmi",
    description="Compute BMI given height (cm) and weight (kg).",
)

# --------------------------------------------------------------------------
# INPUTS: Update these values to test different users
# --------------------------------------------------------------------------
user_profile = {
    "weight_kg": 72,
    "height_cm": 170,
    "age": 29,
    "gender": "Male",
    "dietary_preference": "Veg",  # Veg | Non-Veg | Vegan
}

# Messages for each step in the sequential conversation
messages = [
    (
        "Collect the following user data and proceed with BMI calculation. "
        f"User data: {user_profile}. "
        "Use the calculate_bmi tool and provide BMI score, category, and health recommendations."
    ),
    (
        "Create a tailored 7-day meal plan using the BMI insights and the user's dietary preference. "
        f"Dietary preference: {user_profile['dietary_preference']}."
    ),
    (
        "Create a weekly workout schedule using the meal plan suggestions and the user's details. "
        f"User age: {user_profile['age']}, gender: {user_profile['gender']}."
    ),
]

# Group chat pipeline (round-robin enforces BMI -> Meal -> Workout)
groupchat = GroupChat(
    agents=[
        health_user_proxy,
        bmi_agent,
        meal_planner_agent,
        workout_scheduler_agent,
    ],
    messages=[],
    max_round=4,
    speaker_selection_method="round_robin",
)

groupchat_manager = GroupChatManager(
    groupchat=groupchat,
    llm_config=health_llm_config,
)

# Start the group chat with the user profile context
health_user_proxy.initiate_chat(
    groupchat_manager,
    message=messages[0],
    clear_history=True,
    summary_method="last_msg",
    max_turns=4,
)

print("\n" + "=" * 70)
print("✅ SMART HEALTH ASSISTANT OUTPUT")
print("=" * 70)

# Final output is the last non-empty response from the WorkoutSchedulerAgent
final_health_plan = next(
    (
        msg["content"]
        for msg in reversed(groupchat.messages)
        if msg.get("name") == "WorkoutSchedulerAgent" and msg.get("content")
    ),
    None,
)
print(final_health_plan)


HealthUserProxy (to chat_manager):

Collect the following user data and proceed with BMI calculation. User data: {'weight_kg': 72, 'height_cm': 170, 'age': 29, 'gender': 'Male', 'dietary_preference': 'Veg'}. Use the calculate_bmi tool and provide BMI score, category, and health recommendations.

--------------------------------------------------------------------------------

Next speaker: BMIAgent



BMIAgent (to chat_manager):

***** Suggested tool call (call_zs77YQevP6kAEtD1LOpHy3v4): calculate_bmi *****
Arguments: 
{"height_cm":170,"weight_kg":72}
******************************************************************************

--------------------------------------------------------------------------------

Next speaker: HealthUserProxy


>>>>>>>> EXECUTING FUNCTION calculate_bmi...
Call ID: call_zs77YQevP6kAEtD1LOpHy3v4
Input arguments: {'height_cm': 170, 'weight_kg': 72}

>>>>>>>> EXECUTED FUNCTION calculate_bmi...
Call ID: call_zs77YQevP6kAEtD1LOpHy3v4
Input arguments: {'height_cm': 170, 'weight_kg': 72}
Output:
{'height_m': 1.7, 'bmi': 24.91}
HealthUserProxy (to chat_manager):

***** Response from calling tool (call_zs77YQevP6kAEtD1LOpHy3v4) *****
{'height_m': 1.7, 'bmi': 24.91}
**********************************************************************

--------------------------------------------------------------------------------

Next speaker: BMIAgent

BMIAgent (to chat_mana

In [10]:
from IPython.display import Markdown, display
display(Markdown(final_health_plan))

### FINAL Health Plan

#### BMI Insights
- **Calculated BMI:** 24.91
- **BMI Category:** Normal weight (18.5 - 24.9)
- **Health Recommendations:**
  - Maintain a balanced diet rich in vegetables, fruits, whole grains, and lean proteins.
  - Engage in regular physical activity, aiming for at least 150 minutes of moderate exercise per week.
  - Stay hydrated and monitor portion sizes to maintain your weight.
  - Regularly check your health metrics and consult with a healthcare provider for personalized advice.

#### Diet Plan
Here’s a simple 7-day vegetarian meal plan:

### Day 1
- **Breakfast:** Oatmeal topped with banana and a sprinkle of cinnamon
- **Lunch:** Quinoa salad with chickpeas, cucumber, tomatoes, and lemon dressing
- **Snack:** Greek yogurt with honey and mixed berries
- **Dinner:** Stir-fried tofu with mixed vegetables and brown rice

### Day 2
- **Breakfast:** Smoothie with spinach, banana, and almond milk
- **Lunch:** Lentil soup with whole grain bread
- **Snack:** Carrot sticks with hummus
- **Dinner:** Vegetable curry with basmati rice

### Day 3
- **Breakfast:** Whole grain toast with avocado and poached egg
- **Lunch:** Spinach and feta stuffed bell peppers
- **Snack:** Apple slices with almond butter
- **Dinner:** Pasta with marinara sauce and a side salad

### Day 4
- **Breakfast:** Chia seed pudding with coconut milk and mango
- **Lunch:** Black bean tacos with lettuce, salsa, and avocado
- **Snack:** Mixed nuts
- **Dinner:** Grilled vegetable skewers with quinoa

### Day 5
- **Breakfast:** Smoothie bowl with berries, granola, and seeds
- **Lunch:** Caprese salad with fresh basil and balsamic glaze
- **Snack:** Rice cakes with peanut butter
- **Dinner:** Vegetable stir-fry with soba noodles

### Day 6
- **Breakfast:** Pancakes made with whole wheat flour topped with fresh fruit
- **Lunch:** Falafel wrap with tahini sauce and mixed greens
- **Snack:** Celery sticks with cream cheese
- **Dinner:** Stuffed zucchini boats with brown rice and cheese

### Day 7
- **Breakfast:** Yogurt parfait with granola and seasonal fruits
- **Lunch:** Vegetable sushi rolls with soy sauce
- **Snack:** Popcorn seasoned with nutritional yeast
- **Dinner:** Ratatouille with crusty whole grain bread

### Notes:
- Ensure to drink plenty of water throughout the day.
- Adjust portion sizes based on individual energy needs.
- Feel free to swap meals between days for variety.

#### Weekly Workout Schedule
**Monday:** 
- 30 minutes of brisk walking or jogging
- 15 minutes of bodyweight exercises (push-ups, squats, lunges)

**Tuesday:** 
- Rest day

**Wednesday:** 
- 45 minutes of cycling or swimming

**Thursday:** 
- 30 minutes of yoga or Pilates

**Friday:** 
- 30 minutes of strength training (focus on major muscle groups)

**Saturday:** 
- 30 minutes of hiking or outdoor activity

**Sunday:** 
- Rest day

TERMINATE.